# NanoLLM Training on Kaggle

这个 notebook 从 GitHub 仓库拉取 nano-llm 最新代码，并使用 TinyStories Narrative Classification 数据集进行训练。

## 1. 设置环境变量

从 Kaggle 的环境中获取 HuggingFace 和 Weights & Biases 的 API 密钥

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

# 获取环境变量
user_secrets = UserSecretsClient()
try:
    secret_value_0 = user_secrets.get_secret("HF_TOKEN")
    os.environ["HF_TOKEN"] = secret_value_0
    print("✅ HF_TOKEN 已设置")
except:
    print("⚠️  HF_TOKEN 未找到")

try:
    secret_value_1 = user_secrets.get_secret("WANDB_API_KEY")
    os.environ["WANDB_API_KEY"] = secret_value_1
    print("✅ WANDB_API_KEY 已设置")
except:
    print("⚠️  WANDB_API_KEY 未找到")

print(f"\n环境变量配置完成")

## 2. 安装依赖

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q datasets>=2.10.0 transformers>=4.30.0 tokenizers>=0.13.0 hf-transfer>=0.1.9 huggingface-hub>=0.36.1 kaggle>=1.7.4.5 wandb>=0.24.2

## 3. 从 GitHub 克隆 nano-llm 仓库

In [ ]:
import os
import subprocess

# 克隆仓库
repo_url = "https://github.com/try-agaaain/nano-llm.git"
repo_dir = "/kaggle/working/nano-llm"

if not os.path.exists(repo_dir):
    print(f"正在从 GitHub 克隆仓库: {repo_url}")
    os.system(f"git clone {repo_url} {repo_dir}")
    print(f"✅ 仓库已克隆到: {repo_dir}")
else:
    print(f"仓库已存在于: {repo_dir}")

# 显示仓库内容
print("\n仓库结构:")
for root, dirs, files in os.walk(repo_dir):
    level = root.replace(repo_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # 只显示前5个文件
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... ({len(files) - 5} 更多文件)')
    if level >= 2:  # 限制深度
        dirs.clear()

## 4. 检查数据集

In [ ]:
import os
import pandas as pd

# Kaggle 数据集路径
train_csv = "/kaggle/input/tinystories-narrative-classification/train.csv"
val_csv = "/kaggle/input/tinystories-narrative-classification/validation.csv"

# 检查文件
print("数据集文件检查:")
print(f"  Train CSV: {'✅ 存在' if os.path.exists(train_csv) else '❌ 不存在'} - {os.path.getsize(train_csv) / 1024 / 1024:.2f} MB" if os.path.exists(train_csv) else f"  Train CSV: ❌ 不存在")
print(f"  Val CSV: {'✅ 存在' if os.path.exists(val_csv) else '❌ 不存在'} - {os.path.getsize(val_csv) / 1024 / 1024:.2f} MB" if os.path.exists(val_csv) else f"  Val CSV: ❌ 不存在")

# 显示数据样本
if os.path.exists(train_csv):
    print("\n训练数据集样本:")
    df_train = pd.read_csv(train_csv)
    print(f"  行数: {len(df_train)}")
    print(f"  列: {df_train.columns.tolist()}")
    print(f"\n前3行预览:")
    print(df_train.head(3))

if os.path.exists(val_csv):
    print("\n验证数据集样本:")
    df_val = pd.read_csv(val_csv)
    print(f"  行数: {len(df_val)}")
    print(f"  列: {df_val.columns.tolist()}")

## 5. 运行训练脚本

In [ ]:
import sys
import os

# 切换到 nano-llm 目录
repo_dir = "/kaggle/working/nano-llm"
os.chdir(repo_dir)

# 添加到 Python 路径
sys.path.insert(0, repo_dir)

print(f"当前工作目录: {os.getcwd()}")
print(f"Python 路径已更新")

# 确保环境变量已设置
print(f"\n环境变量检查:")
print(f"  HF_TOKEN: {'✅ 已设置' if os.getenv('HF_TOKEN') else '❌ 未设置'}")
print(f"  WANDB_API_KEY: {'✅ 已设置' if os.getenv('WANDB_API_KEY') else '❌ 未设置'}")

In [ ]:
import subprocess
import sys
import os

# 确保在正确的目录
repo_dir = "/kaggle/working/nano-llm"
os.chdir(repo_dir)

print("启动训练脚本...\n")
print("=" * 70)

# 运行 train.py
result = subprocess.run(
    [sys.executable, "train.py"],
    cwd=repo_dir,
    env={**os.environ},
    capture_output=False,
    text=True
)

print("=" * 70)
print(f"\n训练脚本执行完成，退出码: {result.returncode}")

## 6. 检查输出结果

In [ ]:
import os
import glob

repo_dir = "/kaggle/working/nano-llm"

# 检查模型文件
print("模型输出文件:")
model_files = glob.glob(os.path.join(repo_dir, "*.pt"))
for file in model_files:
    size = os.path.getsize(file) / 1024 / 1024
    print(f"  {os.path.basename(file)}: {size:.2f} MB")

# 检查 tokenizer
print("\nTokenizer 文件:")
tokenizer_dir = os.path.join(repo_dir, "tokenizer")
if os.path.exists(tokenizer_dir):
    for file in os.listdir(tokenizer_dir):
        file_path = os.path.join(tokenizer_dir, file)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path) / 1024
            print(f"  {file}: {size:.2f} KB")

print("\n✅ 训练完成！模型和 tokenizer 已保存。")